# 설문조사 관련 문자 split_0407

- 한 문항에 여러 응답에 대한 분리
- , 로 구성
- str.split(',', expand=True) 를 사용 
- expand=True로 설정 시 col 확장 가능

- 추후 단계 -> 진행(0407
    - 설문 번호 반복문 작성
    - 컬럼 정렬
        - 현재 sort 사용하면 문자로 인식 되어 1과 10이 구분이 안됨

In [ ]:
# 패키지 설치. 안되어 있으면 주석 제거
# !pip install natsort==3.3.0

In [ ]:
#  데이터 전처리
import warnings
import pandas as pd

# 정렬
from natsort import natsorted
# jupyter notebook 화면 조정
# 참조
# https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html

from IPython.display import display

#pd.set_option('display.height', 500)
#pd.set_option('display.max_rows', 500)

pd.set_option('display.max_columns', None)  # 컬럼수 늘려주는 명령
####### warnings.filterwarnings("ignore") # 경고 안나오게 하는 명령
# with pd.option_context('display.max_rows', 100, 'display.max_columns', 100):
#    display(df) #need display to show the dataframe when using with in jupyter
# some pandas stuff

In [ ]:
#  파일 불러오기
df = pd.read_excel('Survey.xlsx', sheet_name='Pet_diagnostic_..._1', skiprows=[
                   1, 2])  # 첫번째 두변째 열은 질문이라 skip
df.head(1)

## 항목 분리

- join 이용하여 원 자료에 붙이는 방식
- split 이용하여 분리
    - 시작이 0번 부터이니 주의
    - **1번부터 시작하고 싶으면 다른 방법 사용해야함** <- 이 방법 추천
- drop하여 원 컬럼 삭제

- 원하는 문항들 넣어서 변경 

- 원래 방법
    - 0부터 시작해서 다른 방법 찾음

In [ ]:
df_m1 = df.join(df['Q_1_2'].str.split(',', expand=True).add_prefix('Q_1_2_'))
df_m1 = df_m1.join(df['Q_8_1'].str.split(
    ',', expand=True).add_prefix('Q_8_1_'))
df_m1 = df_m1.join(df['Q_20_1'].str.split(
    ',', expand=True).add_prefix('Q_20_1_'))
df_m1 = df_m1.drop(['Q_1_2', 'Q_8_1', 'Q_20_1'], axis=1)
df_m1

- 다른 방법
    - rename과 lambda 사용하여 int 형태로 작성하여 1부터 시작

In [ ]:
df_m2 = df.join(df['Q_1_2'].str.split(',', expand=True).rename(
    columns=lambda x: f"Q_1_2_{int(x)+1}"))
df_m2 = df_m2.join(df['Q_8_1'].str.split(',', expand=True).rename(
    columns=lambda x: f"Q_8_1_{int(x)+1}"))
df_m2 = df_m2.join(df['Q_20_1'].str.split(',', expand=True).rename(
    columns=lambda x: f"Q_20_1_{int(x)+1}"))
############# 실제 파일 적용시에는 drop 주석 제거 하고 돌리면 됨 ###################
# df_m2 = df_m2.drop(['Q_1_2', 'Q_8_1', 'Q_20_1'], axis=1)
####################################################################################
df_m2

## 컬럼 정렬
- 설문 관련 정보
- 설문 문항 정보

- 두가지가 섞여 있어 분리하고 설문 문항만 정렬하고 합침

- 설문 문항 중복으로 처음에 .으로 읽혀지는 것들을 _로 변환
    - str.replace보다 깔끔한 방법이 있을듯

In [ ]:
# iloc 사용하여 필요한 컬럼 분리
# https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#selection-by-position

# 설문 관련 정보 분리
df_m2_name = df_m2.iloc[:, :3]
# 설문 문항 정보 분리
df_m2_que = df_m2.iloc[:, 3:]
# 분리 확인
df_m2_que.head(1)

In [ ]:
# 설문 문항 sort
# 설문문항 리스트
df_m2_que_list = df_m2_que.columns.values.tolist()
# 설문 정보 리스트 user id, survey pack 등
df_m2_name_list = df_m2_name.columns.values.tolist()
# 설문문항 sort
df_m2_que_list_sort = natsorted(df_m2_que_list)
# 설문 정보 + 설문 문항 리스트 합치기
df_m2_col_list = df_m2_name_list + df_m2_que_list_sort
# 확인
df_m2_col_list
# 컬럼 재정렬
df_m2 = df_m2[df_m2_col_list]
# . -> _로 변경
df_m2_cha = df_m2.reindex(
    (df_m2.columns.to_series().str.replace(".", "_")), axis=1)
# 확인
df_m2_cha
# [sub.replace('.','_') for sub in col_sorted]
# natsorted(col_sort_data)

- 파일 저장 
    - 기존 시트에 덮어 씌우는 것도 가능

In [ ]:
# 덮어쓸 경우 해당 시트 삭제 하고 다시 작성
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_excel.html
with pd.ExcelWriter('Survey.xlsx',
                    mode='a') as writer:
    # df_m2_cha 에서 데이터프레임 입력
    df_m2_cha.to_excel(
        writer, sheet_name='Pet_diagnostic_preprocess', index=False)

- 끝?

---------------------------------------
나머지는 이런 저런 작업

In [ ]:
collist = df_m2.columns.values.tolist()
collist

In [ ]:
collist[:3]

In [ ]:
col_sort_data = collist[3:]
col_sorted = natsorted(col_sort_data)
col_sorted = [sub.replace('.', '_') for sub in col_sorted]
col_sorted

In [ ]:
# df = pd.read_csv('test.csv', skiprows=[1,2])

In [ ]:
# 컬럼 이름 확인
# 나눌 때 참고 하기 위함
df.columns

## 항목 분리

- 사이트
- https://pandas.pydata.org/docs/reference/api/pandas.Series.str.split.html 
- str.split 사용
- expand= True 사용하여 컬럼 분리
- 설문명.columns 사용하여 컬럼명 표기

- 추가
    - 컬럼명 반복문으로 작성 가능할듯...?
    - 찾아보면 나올듯............................

In [ ]:
Q_1_2 = df["Q_1_2"].str.split(',', expand=True)
Q_1_2

In [ ]:
# 컬럼명 작성을 위해 개수 파악
len(Q_1_2.columns)

In [ ]:
# 개수 대로 세부 문항 작성
# 나중에 list comprehension apply로 대체 가능?
# suffix 처리하면 뒤에 _1 _2 붙을듯
Q_1_2.columns = ['Q_1_2_1', 'Q_1_2_2', 'Q_1_2_3']

In [ ]:
Q_1_2

In [ ]:
# concat으로 합침 - 마지막에 합쳐도 가능~~
# df = pd.concat([df, Q_1_2], axis = 1)
# df.head()

- 8번 문항 샘플

In [ ]:
Q_8_1 = df["Q_8_1"].str.split(',', expand=True)
len(Q_8_1.columns)

In [ ]:
Q_8_1.columns = ["Q_8_1_1", "Q_8_1_2", "Q_8_1_3", "Q_8_1_4",
                 "Q_8_1_5", "Q_8_1_6", "Q_8_1_7", "Q_8_1_8", "Q_8_1_9"]

- 데이터 프레임 합치기

    - link1 : https://pandas.pydata.org/docs/user_guide/merging.html#more-concatenating-with-group-keys
    - link2 : https://m.blog.naver.com/youji4ever/221731555544

In [ ]:
# 데이터 프레임 여러개 가능!

df = pd.concat([df, Q_1_2, Q_8_1], axis=1)
df.head()
# sample.columns = ['Q_1_2_1', 'Q_1_2_2', 'Q_1_2_3']

## 컬럼 순서 정렬하여 번호 순으로 메기

In [ ]:
collist = df.columns.values.tolist()
sorted(collist, reverse=False)

In [ ]:
collist[3:]

In [ ]:
from natsort import natsorted

In [ ]:
natsorted(collist)

In [ ]:
natsorted(collist[3:])